* Criado por: Gabriel Quintella
* Data Criação: 24.01.2023
* Data Última Modificação: 29.01.2023
* Repositório: https://github.com/dbaassists/Projeto-KB-Spark/blob/main/Projeto%20KB%20Spark.ipynb

DESCRIÇÃO DO PROJETO

Criar uma base de conhecimento (KB) com comandos pyspark.

*  Dia 1 - 24.01.2023 - Lançamento do Projeto;
*  Dia 2 - 26.01.2023 - Aprimoramento do Projeto com melhoria nos exemplos criados;
*  Dia 3 - 27.01.2023 - Aprimoramento do Projeto com melhoria nos exemplos criados;
*  Dia 4 - 28.01.2023 - Aprimoramento do Projeto com melhoria nos exemplos criados;
*  Dia 5 - 29.01.2023 - Aprimoramento do Projeto com melhoria nos exemplos criados;



 Sumário

* 01 - Instalação da Biblioteca PySpark
* 02 - Definição de Bibliotecas
* 03 - Importando Arquivo CSV
* 04 - Transformando o DataFrame Spark em Tabela Temporária
* 05 - Consultando a Estrutura do DataFrame Spark
* 06 - Lendo a Tabela e Transformando em outro DataFrame Spark
* 07 - Convertendo o DataType de uma Coluna de um DataFrame Spark
* 08 - Convertendo o DataType e Criando uma nova Coluna de um DataFrame Spark
* 09 - Renomeando uma Coluna em DataFrame Spark
* 10 - Renomeando Várias Colunas em DataFrame Spark
* 11 - Ordenando um DataFrame Spark


# 01 - Instalação da Biblioteca PySpark

In [ ]:
#pip install pyspark

# 02 - Definição de Bibliotecas

In [ ]:
import sys
import requests
import pandas as pd
import json
import pyspark.sql.functions as F #import explode_outer, col
from pyspark.sql.types import *
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark import SparkConf
from pyspark import SparkContext
import urllib
from sqlalchemy import __version__ as sa_version, create_engine, text


# 03 - Importando Arquivo CSV

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

arquivo = '../ProjetoSparkKB/Dados/Candidato/consulta_cand_2022_BRASIL.csv'

df = spark.read.csv(arquivo, sep=';',header=True)

#df.select('DT_GERACAO').show()

df.show(2,truncate=False)

# 03.1 - Importando Arquivo CSV

In [ ]:
arquivo = '../ProjetoSparkKB/Dados/Candidato/consulta_cand_2022_BRASIL.csv'

dfExemplo2 = spark.read.format("csv").load(arquivo)
    
dfExemplo2.printSchema()

# 03.2 - Importando Arquivo CSV

In [ ]:
arquivo = '../ProjetoSparkKB/Dados/Candidato/consulta_cand_2022_BRASIL.csv'

dfExemplo3 = spark.read.option("inferSchema",False) \
                .option("delimiter",";") \
                .option("header",True) \
                .csv(arquivo)

dfExemplo3.printSchema()

# 03.3 - Importando Arquivo CSV

In [ ]:
arquivo = '../ProjetoSparkKB/Dados/Candidato/consulta_cand_2022_BRASIL.csv'

dfExemplo04 = spark.read.options(header='True', inferSchema='True', delimiter=';') \
  .csv(arquivo)

dfExemplo04.printSchema()

# 03.4 - Importando Arquivo CSV

In [ ]:
arquivo = '../ProjetoSparkKB/Dados/IndicadorSegurancaPublica/indicador_homocidio.csv'

df3 = spark.read.options(header='True', inferSchema='True', delimiter=';').csv(arquivo)

df3.show()

# 03.5 - Importando Arquivo CSV

In [ ]:
arquivo = '../ProjetoSparkKB/Dados/IndicadorSegurancaPublica/indicador_homocidio.csv'

df3 = spark.read.options(header='True', inferSchema='True', delimiter=';').csv(arquivo)

df3.show(3)

# 03.6 - Importando Arquivo CSV

** Documentação: https://sqlrelease.com/spark-read-file-with-special-characters-using-pyspark

** Documentação: https://sparkbyexamples.com/pyspark/pyspark-sql-types-datatype-with-examples/

** Documentação: https://spark.apache.org/docs/latest/sql-ref-datatypes.html

** Documentação: https://sparkbyexamples.com/pyspark/pyspark-date_format-convert-date-to-string-format/

In [ ]:
arquivo = '../ProjetoSparkKB/Dados/IndicadorSegurancaPublica/indicador_homocidio.csv'


schema = StructType() \
      .add("CodUf",StringType(),True) \
      .add("TipoCrime",StringType(),True) \
      .add("Quantidade",IntegerType(),True) \
      .add("Data",DateType(),True) \


df_with_schema = spark.read.options(header='True', inferSchema='False', delimiter=';', encoding="windows-1252") \
      .schema(schema) \
      .csv(arquivo)

df_with_schema.printSchema()

df_with_schema.show()

# 04 - Trabalhando com Datas - Converte coluna String (DD/MM/AAA) para Date (AAAAMMDD)

* Trabalhando com Datas

** Documentação: https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import StringType

arquivo = '../ProjetoSparkKB/Dados/IndicadorSegurancaPublica/indicador_homocidio_2.csv'

schema = StructType() \
      .add("CodUf",StringType(),True) \
      .add("TipoCrime",StringType(),True) \
      .add("Quantidade",IntegerType(),True) \
      .add("Data",StringType(),True) \

df_with_schema2 = spark.read.options(header='True', inferSchema='False', delimiter=';', encoding="windows-1252") \
      .schema(schema) \
      .csv(arquivo)

df_with_schema2.show()

df_with_schema2 = df_with_schema2.withColumn('Data', to_date(date_format(unix_timestamp(df_with_schema2.Data , "dd/mm/yyyy").cast("timestamp"),"yyyy-mm-dd")))

df_with_schema2.printSchema()

df_with_schema2.show()

# 04.1 - Trabalhando com Datas - A partir de uma coluna de Data, criar outra de Ano

In [ ]:

df_with_schema2 = df_with_schema2.withColumn("Ano", date_format(col("Data"), "y"))


df_with_schema2.printSchema()

# 04.2 - Trabalhando com Datas - A partir de uma coluna de Data, criar outra de Mês

In [ ]:

df_with_schema2 = df_with_schema2.withColumn("NumMes", date_format(col("Data"), "M"))

df_with_schema2 = df_with_schema2.withColumn("NomMes", date_format(col("Data"), "L"))


df_with_schema2.printSchema()

df_with_schema2.show()

# 04.3 - Trabalhando com Datas - A partir de uma coluna de Data, criar outra de Dia

In [ ]:

df_with_schema2 = df_with_schema2.withColumn("NumDiaAno", date_format(col("Data"), "D"))

df_with_schema2 = df_with_schema2.withColumn("NumDiaMes", date_format(col("Data"), "d"))

df_with_schema2.printSchema()

df_with_schema2.show()

# 05 - Trabalhando com Arquivos Parquet

In [ ]:
#arquivoDestino = '../ProjetoSparkKB/Dados/Parquet/indicador_homocidio.parquet'

#df_with_schema3.write.mode("overwrite").parquet(arquivoDestino)

# 06 - Comando Semelhante ao DataFrame.info() do Pandas

In [ ]:
df_with_schema3.summary().show()

# 04 - Transformando o DataFrame Spark em Tabela Temporária

In [ ]:
df.createOrReplaceTempView('teste')

# 05 - Consultando a Estrutura do DataFrame Spark

In [ ]:
df.printSchema()

# 06 - Lendo a Tabela e Transformando em outro DataFrame Spark

In [ ]:
df2 = spark.sql("""
SELECT DT_GERACAO 
FROM TESTE
""")

# 07 - Convertendo o DataType de uma Coluna de um DataFrame Spark

In [ ]:
df2.withColumn("DT_GERACAO", df2["DT_GERACAO"].cast("timestamp")).printSchema()

# 08 - Convertendo o DataType e Criando uma nova Coluna de um DataFrame Spark

In [ ]:
df2.withColumn("DTGERACAO", df2["DT_GERACAO"].cast("timestamp")).printSchema()

# 09 - Renomeando uma Coluna em DataFrame Spark

In [ ]:
df2.withColumnRenamed("DT_GERACAO","DT_GERACAO2").printSchema()

# 10 - Renomeando Várias Colunas em DataFrame Spark

In [ ]:
df3 = spark.sql("""
SELECT DT_GERACAO , HH_GERACAO
FROM TESTE
""")

In [ ]:
df3 = df3.withColumnRenamed("DT_GERACAO","DtGeracao") \
    .withColumnRenamed("HH_GERACAO","HrGeracao")
df3.printSchema()

# 11 - Ordenando um DataFrame Spark

In [ ]:
df4 = spark.sql("""
SELECT SG_UF , NM_URNA_CANDIDATO
FROM TESTE
""")

In [ ]:
df4.show()

In [ ]:
df4.sort("SG_UF").show(truncate=False)


# 11.1 - Ordenando um DataFrame Spark por Várias Colunas

In [ ]:
df4.sort("SG_UF","NM_URNA_CANDIDATO").show(truncate=False)


In [ ]:

df4.sort(col("SG_UF"),col("NM_URNA_CANDIDATO")).show(truncate=False)

# 12 - Agrupando um DataFrame Spark

In [ ]:
df4.groupBy("SG_UF").count().show(truncate=False)

In [ ]:
df5 = spark.sql("""
SELECT SG_UF , NM_URNA_CANDIDATO, SQ_CANDIDATO
FROM TESTE
""")

# 13 - Funções de Agrupamento (Count(), Sum(), Min(), Max() e Mean())

In [ ]:
df5.groupBy("SG_UF") \
    .agg(count("SQ_CANDIDATO").alias("count_SQ_CANDIDATO"), \
         sum("SQ_CANDIDATO").alias("sum_SQ_CANDIDATO"), \
         avg("SQ_CANDIDATO").alias("avg_SQ_CANDIDATO"), \
         min("SQ_CANDIDATO").alias("min_SQ_CANDIDATO"), \
         max("SQ_CANDIDATO").alias("max_SQ_CANDIDATO") \
     ) \
    .show(truncate=False)